# Use result of segment.ipynb to classify


In [1]:
from sklearn.ensemble import AdaBoostClassifier
import PIL.Image as pil
import skops.io as skio
import numpy as np

In [2]:
RASTER_PATH = "../no_subsampling/raster__mnt_Poznan_Internship_Data_Campus_cloud2853085222bfe35e_Block_3.las.tif"
LABEL_PATH = "../no_subsampling/label__mnt_Poznan_Internship_Data_Campus_cloud2853085222bfe35e_Block_3.las.tif"
MODEL_PATH = "./classifier.skops"
MODEL_TYPE = AdaBoostClassifier

In [3]:
raster_img = pil.open(RASTER_PATH)
raster_data = np.array(raster_img.getdata())
label_img = pil.open(LABEL_PATH)
label_data = np.array(label_img.getdata())
model: MODEL_TYPE = skio.load(MODEL_PATH)
model.get_params()

{'algorithm': 'deprecated',
 'estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 50,
 'random_state': None}

In [4]:
labels = np.unique(label_data)
labels

array([   0,    2,    4, ..., 1743, 1744, 1745])

In [7]:
segments_img = dict()
filter_size = 1000

mask = np.zeros(raster_data.shape, dtype=int)

for l in labels:
    mask[label_data == l] = [1, 1, 1]
    result = raster_data * mask
    mask[:, :] = [0, 0, 0]
    seg = np.argwhere(result)
    print(seg.shape)
    if not seg.shape[0] or not seg.shape[1]:
        continue
    if np.sum(seg.shape[:1]) < filter_size:
        continue

    minx, maxx = np.min(seg[:, 0]), np.max(seg[:, 0])
    miny, maxy = np.min(seg[:, 1]), np.max(seg[:, 1])
    segment = result[minx:maxx + 1, miny:maxy + 1]
    print(segment.shape)
    segments_img.update({l: [
        np.mean(segment[:, :, 0]),
        np.mean(segment[:, :, 1]),
        np.mean(segment[:, :, 2]),
        np.median(segment[:, :, 0]),
        np.median(segment[:, :, 1]),
        np.median(segment[:, :, 2])
    ]})

(1616369, 2)
(2679205, 3)


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed